# Identificando usuários que mais retweetam

In [2]:
# Imports
import pandas as pd

In [3]:
data = pd.read_json('tweets_222_23abr2020.json', convert_dates=False, lines = True, chunksize = 1) # ABRINDO O ARQUIVO JSON NO MESMO DIRETÓRIO QUE O IPYNB
retweets = pd.Series(name = 'ID')

num = 0
for i in data:
    if(num<=5000): # NÚMERO DE TWEETS A SEREM LIDOS
        if i['text'].values[0].startswith('RT'): # REPASSANDO PARA "retweets" APENAS OS QUE SE INICIAM COM "RT"
            retweets = retweets.append(i[['user', 'created_at']])
            num += 1
    else:
        break
    
retweets.drop(columns=0)
retweets.size

<ipython-input-3-1256338302a7>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  retweets = pd.Series(name = 'ID')


15003

In [4]:
retweets.head()

,0,created_at,user
1,NaN,{'$date': '2020-04-11T20:40:10.000-0300'},"{'follow_request_sent': None, 'profile_use_bac..."
2,NaN,{'$date': '2020-04-11T20:40:10.000-0300'},"{'follow_request_sent': None, 'profile_use_bac..."
4,NaN,{'$date': '2020-04-11T20:40:10.000-0300'},"{'follow_request_sent': None, 'profile_use_bac..."
5,NaN,{'$date': '2020-04-11T20:40:10.000-0300'},"{'follow_request_sent': None, 'profile_use_bac..."
8,NaN,{'$date': '2020-04-11T20:40:11.000-0300'},"{'follow_request_sent': None, 'profile_use_bac..."


### Criando a coluna "Day"

In [5]:
retweets['Day'] = ""

In [8]:
retweets = retweets.reset_index() # RESETANDO OS INDEXES (INICIANDO EM 0)

In [9]:
num = 0
for i in range(len(retweets.index)):
    retweets.at[num, 'Day'] = retweets['created_at'][num]['$date'].split('T')[0]
    num += 1

In [12]:
retweets.head()

,index,0,created_at,user,Day
0,1,NaN,{'$date': '2020-04-11T20:40:10.000-0300'},"{'follow_request_sent': None, 'profile_use_bac...",2020-04-11
1,2,NaN,{'$date': '2020-04-11T20:40:10.000-0300'},"{'follow_request_sent': None, 'profile_use_bac...",2020-04-11
2,4,NaN,{'$date': '2020-04-11T20:40:10.000-0300'},"{'follow_request_sent': None, 'profile_use_bac...",2020-04-11
3,5,NaN,{'$date': '2020-04-11T20:40:10.000-0300'},"{'follow_request_sent': None, 'profile_use_bac...",2020-04-11
4,8,NaN,{'$date': '2020-04-11T20:40:11.000-0300'},"{'follow_request_sent': None, 'profile_use_bac...",2020-04-11


### Criando a coluna "Username"

In [14]:
retweets['Username'] = ""

In [19]:
# POPULANDO A COLUNA "Username" CONFORME O DICIONÁRIO "user"
num = 0
for i in range(len(retweets.index)):
    retweets.at[num, 'Username'] = retweets['user'][num]['screen_name']
    num += 1

### Coletando dias disponíveis no conjunto de dados

In [20]:
days = retweets['Day'].unique()
print(days)

['2020-04-11']


# Obtendo X usuários que mais tweetaram em um dia Y

In [21]:
x = 3
y = "2020-04-12"

# AVISANDO CASO UM DIA NÃO ESTEJA DISPONÍVEL NO CONJUNTO CARREGADO
if y not in days:
    print(y + ' não está disponível no conjunto de tweets analisados.')
    y = days[0]
    print('Definindo y como o primeiro dia disponível no banco de dados: ' + y)

2020-04-12 não está disponível no conjunto de tweets analisados.
Definindo y como o primeiro dia disponível no banco de dados: 2020-04-11


In [24]:
results = retweets.loc[retweets['Day'] == y].groupby('Username').count().sort_values(by='user', ascending = False)
print(results)

                 index  0  created_at  user  Day
Username                                        
IvaniseShakira      14  0          14    14   14
BrisaBot             9  0           9     9    9
ac7cbf7867e6429      8  0           8     8    8
Andrew_Spartano      8  0           8     8    8
nuncaabandona        7  0           7     7    7
...                ... ..         ...   ...  ...
Tales_Quina          1  0           1     1    1
Tamirandaa1          1  0           1     1    1
Tata_nasc1           1  0           1     1    1
TatiPampy            1  0           1     1    1
zzzzxxxxccccaa       1  0           1     1    1

[4262 rows x 5 columns]


#### Resultado:

In [25]:
results.head(x)

,index,0,created_at,user,Day
Username,,,,,
IvaniseShakira,14,0,14,14,14
BrisaBot,9,0,9,9,9
ac7cbf7867e6429,8,0,8,8,8


### Obtendo o usuário que mais retweetou nesse dia Y

In [26]:
results['user'].idxmax()

'IvaniseShakira'